In [2]:
# Libraries
import os
import re
import json
import folium
import pandas as pd
import matplotlib.pyplot as plt
from urllib.request import urlopen


# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

d:\zhaw_wirtschaftsinformatik\Semester_4\SP\LN
